# CTA data analysis with Gammapy

## Introduction

**This notebook shows an example how to make a sky image and spectrum for simulated CTA data with Gammapy.**

The dataset we will use is three observation runs on the Galactic center. This is a tiny (and thus quick to process and play with and learn) subset of the simulated CTA dataset that was produced for the first data challenge in August 2017.


## Setup

As usual, we'll start with some setup ...

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import MatplotlibDeprecationWarning

# this in general not advised, but for this specific notebook
# I'd like to ignore some deprecation warnings by matplotlib
import warnings
warnings.filterwarnings(
    "ignore", category=MatplotlibDeprecationWarning
)

In [ ]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.convolution import Gaussian2DKernel
from regions import CircleSkyRegion
from gammapy.modeling import Fit
from gammapy.data import DataStore
from gammapy.datasets import (
    Datasets,
    FluxPointsDataset,
    SpectrumDataset,
    MapDataset,
)
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    SkyModel,
    GaussianSpatialModel,
)
from gammapy.maps import MapAxis, WcsNDMap, WcsGeom
from gammapy.makers import (
    MapDatasetMaker,
    SafeMaskMaker,
    SpectrumDatasetMaker,
    ReflectedRegionsBackgroundMaker,
)
from gammapy.estimators import TSMapEstimator, FluxPointsEstimator
from gammapy.estimators.utils import find_peaks
from gammapy.visualization import plot_spectrum_datasets_off_regions

## Select observations

A Gammapy analysis usually starts by creating a `~gammapy.data.DataStore` and selecting observations.

This is shown in detail in the other notebook, here we just pick three observations near the galactic center.

In [ ]:
data_store = DataStore.from_dir("data/cta-1dc/index/gps")

In [ ]:
obs_id = [110380, 111140, 111159]
observations = data_store.get_observations(obs_id)

In [ ]:
obs_cols = ["OBS_ID", "GLON_PNT", "GLAT_PNT", "LIVETIME"]
data_store.obs_table.select_obs_id(obs_id)[obs_cols]

## Make sky images

### Define map geometry

Select the target position and define an ON region for the spectral analysis

In [ ]:
axis = MapAxis.from_energy_bounds("0.1 TeV", "10 TeV", nbin=5)

geom = WcsGeom.create(
    skydir=(0, 0),
    npix=(500, 400),
    binsz=0.02,
    frame="galactic",
    axes=[axis]
)
geom

### Compute Images

To compute images we rely on the classe defined in `gammapy.makers`

In [ ]:
%%time
stacked = MapDataset.create(geom=geom)
maker = MapDatasetMaker()
maker_safe_mask = SafeMaskMaker(
    methods=["offset-max"], offset_max=2.5 * u.deg
)

for obs in observations:
    cutout = stacked.cutout(obs.pointing_radec, width="5 deg")
    dataset = maker.run(cutout, obs)
    dataset = maker_safe_mask.run(dataset, obs)
    stacked.stack(dataset)

In [ ]:
# The maps are cubes, with an energy axis.
# Let's also make some images:
dataset_image = stacked.to_image()

### Show images

Let's have a quick look at the images we computed ...

In [ ]:
dataset_image.counts.smooth(2).plot(vmax=5, cmap="inferno");

In [ ]:
dataset_image.background.plot(vmax=5, cmap="inferno");

In [ ]:
dataset_image.excess.smooth(3).plot(vmax=2, cmap="inferno");

## Source Detection

Use the class `~gammapy.estimators.TSMapEstimator` and function `gammapy.estimators.utils.find_peaks` to detect sources on the images. We search for 0.1 deg sigma gaussian sources in the dataset.

In [ ]:
spatial_model = GaussianSpatialModel(sigma="0.05 deg")
spectral_model = PowerLawSpectralModel(index=2)
model = SkyModel(spatial_model=spatial_model, spectral_model=spectral_model)

In [ ]:
ts_image_estimator = TSMapEstimator(
    model,
    kernel_width="0.5 deg",
    selection_optional=[],
    downsampling_factor=2,
    sum_over_energy_groups=False,
    energy_edges=[0.1, 10] * u.TeV,
)

In [ ]:
%%time
images_ts = ts_image_estimator.run(stacked)

In [ ]:
images_ts["sqrt_ts"].plot(add_cbar=True, cmap="inferno");

In [ ]:
sources = find_peaks(
    images_ts["sqrt_ts"].get_image_by_idx((0,)),
    threshold=5,
    min_distance="0.2 deg",
)
sources

In [ ]:
# Plot sources on top of significance sky image
images_ts["sqrt_ts"].plot(add_cbar=True, cmap="inferno")

plt.gca().scatter(
    sources["ra"],
    sources["dec"],
    transform=plt.gca().get_transform("icrs"),
    color="none",
    edgecolor="white",
    marker="o",
    s=200,
    lw=1.5,
);

## Spectal Analysis

We'll run a spectral analysis using the classical reflected regions background estimation method,
and using the on-off (often called WSTAT) likelihood function.

In [ ]:
region = CircleSkyRegion(
    center=SkyCoord(0, 0, unit="deg", frame="galactic"),
    radius = 0.2 * u.deg
)

e_reco = MapAxis.from_energy_bounds(
    0.1, 40, 40, unit="TeV", name="energy"
)

e_true = MapAxis.from_energy_bounds(
    0.05, 100, 200, unit="TeV", name="energy_true"
)

dataset_empty = SpectrumDataset.create(
    e_reco=e_reco, e_true=e_true, region=region
)

In [ ]:
geom_image = geom.to_image()

data = geom_image.region_mask([region], inside=False)
exclusion_mask = WcsNDMap(geom_image, data=data)
exclusion_mask.plot();

In [ ]:
dataset_maker = SpectrumDatasetMaker(
    containment_correction=False,
    selection=["counts", "exposure", "edisp"]
)

bkg_maker = ReflectedRegionsBackgroundMaker(
    exclusion_mask=exclusion_mask
)

safe_mask_masker = SafeMaskMaker(
    methods=["aeff-max"], aeff_percent=10
)

In [ ]:
%%time
datasets = Datasets()

for observation in observations:
    dataset = dataset_maker.run(
        dataset_empty.copy(name=f"obs-{observation.obs_id}"),
        observation
    )
    dataset_on_off = bkg_maker.run(dataset, observation)
    dataset_on_off = safe_mask_masker.run(dataset_on_off, observation)
    datasets.append(dataset_on_off)

In [ ]:
plt.figure(figsize=(8, 8))
_, ax, _ = dataset_image.counts.smooth("0.03 deg").plot(vmax=8, cmap="inferno")

region.to_pixel(ax.wcs).plot(ax=ax, edgecolor="white")
plot_spectrum_datasets_off_regions(datasets, ax=ax)

### Model fit

The next step is to fit a spectral model, using all data (i.e. a "global" fit, using all energies).

In [ ]:
%%time
spectral_model = PowerLawSpectralModel(
    index=2, amplitude=1e-11 * u.Unit("cm-2 s-1 TeV-1"), reference=1 * u.TeV
)

model = SkyModel(spectral_model=spectral_model, name="source-gc")

datasets.models = model

fit = Fit(datasets)
result = fit.run()
print(result)

### Spectral points

Finally, let's compute spectral points. The method used is to first choose an energy binning, and then to do a 1-dim likelihood fit / profile to compute the flux and flux error.

In [ ]:
# Flux points are computed on stacked observation
stacked_dataset = datasets.stack_reduce(name="stacked")

print(stacked_dataset)

In [ ]:
energy_edges = MapAxis.from_energy_bounds("1 TeV", "30 TeV", nbin=5).edges

stacked_dataset.models = model

fpe = FluxPointsEstimator(energy_edges=energy_edges, source="source-gc")
flux_points = fpe.run(datasets=[stacked_dataset])
flux_points.table_formatted

### Plot

Let's plot the spectral model and points. You could do it directly, but for convenience we bundle the model and the flux points in a `FluxPointDataset`:

In [ ]:
flux_points_dataset = FluxPointsDataset(data=flux_points, models=model)

In [ ]:
flux_points_dataset.plot_fit();